# Sea Floor Objects

In [34]:
# This will grow and change.
import os
import earthpy as et
import pandas as pd
import geopandas as gpd
from io import BytesIO
import numpy as np
import requests
from shapely.geometry import Point, Polygon
from fiona.drvsupport import supported_drivers


In [35]:
# Download the google earth KML data from dbSeabed
et.data.get_data(
    url=("http://instaar.colorado.edu/~jenkinsc"
         "/dbseabed/kml/dbS_global.kml"))

'/Users/robynmarowitz/earth-analytics/data/earthpy-downloads/dbS_global.kml'

In [36]:
# go to /earth-analytics/data directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

# Define the path where this data will be held
dbseabed_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'dbS_global.kml')

In [4]:
# Artifical reefs data download

# r = requests.get("https://deq.nc.gov/marine-fisheries/coastal-fishing-"
#                 "information/artificial-reefs/reef-kmz-file/open")


# r = BytesIO(r.content)

In [5]:
# NOTE: Download instructions are in the README for this.
reef_kml_pth = os.path.join(
   et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
   'Reef_Material.kml')

In [16]:
# Get datasets from the Wrecks and Obstructions Database
ENC_wrecks_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'ENC_wrecks.kml')
AWOIS_wrecks_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'AWOIS_Obstructions.kml')

# Caching
if not os.path.exists(ENC_wrecks_kml_pth):
    et.data.get_data(
        url=("https://wrecks.nauticalcharts.noaa.gov/"
         "downloads/ENC_Wrecks.kml"))
if not os.path.exists(AWOIS_wrecks_kml_pth):  
    et.data.get_data(
        url="https://wrecks.nauticalcharts.noaa.gov/"
        "downloads/AWOIS_Obstructions.kml")



In [17]:
# TODO: we need to create a bounding box for the northern coastline of 
# north carolina and the souther coastline of south carolina and decide 
# how much ocean to include

NC_northeast_land_point = (36.541466, -77.121369)
SC_southest_land_point = (32.087495, -81.037622)
NC_water_point = (36.545849, -71.760165)
SC_water_point = (32.368586, -71.511922)
bounding_coords = [(36.541466, -77.121369), 
                   (36.541466, -71.760165), 
                   (32.087495, -71.511922),
                   (31.036502, -78.317663)]

box = {'geometry': [Polygon([(36.541466, -77.121369), 
                   (40.541466, -70.760165), 
                   (32.087495, -71.511922),
                   (31.036502, -78.317663)])]}
bbox_gdf = gpd.GeoDataFrame(box, crs='EPSG:4326')
bbox_gdf.bounds


,minx,miny,maxx,maxy
0,31.036502,-78.317663,40.541466,-70.760165


In [18]:
# Read in the KML and trim it to north and south carolina
def read_kml(kml_fp: str):
    """Read in a KML file to a geopandas dataframe.
    
    Inputs
    -------
    kml_fp: str
        filepath where KML is located
    Outputs:
    --------
    gdf: gpd.GeoDataFrame
    """
    
    supported_drivers['KML'] = 'rw'
    gdf = gpd.read_file(
        kml_fp, driver='KML')
        # bbox=(31.036502,-78.317663,40.541466,-71.511922))
    return gdf

# Turn geometry into Lattitude and longitude, create desired format og
# Lat, Lon, Size, Description
# Need to decide if I should still keep geometry as a 5th row?
# We may want this to change to something like "create_refined_gdf"
def create_refined_df(gdf: gpd.GeoDataFrame):
    fgdf = gpd.GeoDataFrame(
        columns=['lat', 'lon', 'size', 'description', 'geometry'])
    fgdf['lon'] = gdf.geometry.x
    fgdf['lat'] = gdf.geometry.y
    fgdf['geometry'] = gdf.geometry
    return fgdf



In [19]:
# Figure out what is going on here...why is it empty?
dbS_gdf = read_kml(dbseabed_kml_pth)
type(dbS_gdf)
dbS_gdf

,Name,Description,geometry


In [20]:
# ENC Wrecks gdf
ENC_gdf = read_kml(ENC_wrecks_kml_pth)
ENC_gdf


,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (-79.03783 9.56900 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (-78.87901 9.55749 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (-78.94357 9.55448 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (-72.54199 18.23128 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (-72.53418 18.22828 0.00000)
...,...,...,...
11490,kml_11491,<center><table><tr><th colspan='2' align='cent...,POINT Z (-87.21964 18.70609 0.00000)
11491,kml_11492,<center><table><tr><th colspan='2' align='cent...,POINT Z (-86.65335 21.26916 0.00000)
11492,kml_11493,<center><table><tr><th colspan='2' align='cent...,POINT Z (-170.65833 14.37500 0.00000)
11493,kml_11494,<center><table><tr><th colspan='2' align='cent...,POINT Z (-117.39005 32.26052 0.00000)


In [21]:
# Convert ENC gdf to refined df
ENC_df = create_refined_df(ENC_gdf)
ENC_df['description'] = 'ENC Wreck'
ENC_df

,lat,lon,size,description,geometry
0,9.569000,-79.037834,NaN,ENC Wreck,POINT Z (-79.03783 9.56900 0.00000)
1,9.557486,-78.879013,NaN,ENC Wreck,POINT Z (-78.87901 9.55749 0.00000)
2,9.554478,-78.943573,NaN,ENC Wreck,POINT Z (-78.94357 9.55448 0.00000)
3,18.231279,-72.541992,NaN,ENC Wreck,POINT Z (-72.54199 18.23128 0.00000)
4,18.228279,-72.534180,NaN,ENC Wreck,POINT Z (-72.53418 18.22828 0.00000)
...,...,...,...,...,...
11490,18.706093,-87.219643,NaN,ENC Wreck,POINT Z (-87.21964 18.70609 0.00000)
11491,21.269161,-86.653351,NaN,ENC Wreck,POINT Z (-86.65335 21.26916 0.00000)
11492,14.375000,-170.658333,NaN,ENC Wreck,POINT Z (-170.65833 14.37500 0.00000)
11493,32.260523,-117.390053,NaN,ENC Wreck,POINT Z (-117.39005 32.26052 0.00000)


In [22]:
# AWOIS Wrecks gdf
AWOIS_gdf = read_kml(AWOIS_wrecks_kml_pth)
AWOIS_gdf


# df
AWOIS_df = create_refined_df(AWOIS_gdf)
AWOIS_df['description'] = 'AWOIS Obstruction'
# AWOIS_df1 = AWOIS_df.clip((31.036502,-78.317663, 40.541466,-71.511922))
# AWOIS_df1


In [30]:
# reef material
reef_gdf = read_kml(reef_kml_pth)
reef_gdf
# reproject to eastern US zone
# EPSG 

,Name,Description,geometry
0,Granite,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45228 35.09324 0.00000, ..."
1,Granite,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45232 35.09293 0.00000, ..."
2,Granite,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45224 35.09274 0.00000, ..."
3,Granite,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45246 35.09263 0.00000, ..."
4,Granite,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45230 35.09245 0.00000, ..."
...,...,...,...
14352,Marl,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45295 35.09328 0.00000, ..."
14353,Marl,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45294 35.09318 0.00000, ..."
14354,Marl,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45313 35.09353 0.00000, ..."
14355,Marl,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-76.45311 35.09367 0.00000, ..."


In [32]:
# TODO: Figure out why the fuck I cannot get the clipping to work.
reef_clip = gpd.clip(reef_gdf, (31.036502,-78.317663, 40.541466,-71.511922))
reef_clip

,Name,Description,geometry


In [33]:

reef_df = pd.DataFrame(columns=['lat', 'lon', 'size', 'description', 'geometry'])
reef_df['lon'] = reef_gdf.geometry.centroid.x
reef_df['lat'] = reef_gdf.geometry.centroid.y
reef_df['description'] = reef_gdf.Name
reef_df['geometry'] = reef_gdf.geometry
reef_df

/var/folders/8m/cwkysk5d759d5nwvy056cdxw0000gn/T/ipykernel_15833/3520165020.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reef_df['lon'] = reef_gdf.geometry.centroid.x
/var/folders/8m/cwkysk5d759d5nwvy056cdxw0000gn/T/ipykernel_15833/3520165020.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reef_df['lat'] = reef_gdf.geometry.centroid.y


,lat,lon,size,description,geometry
0,35.093102,-76.451920,NaN,Granite,"MULTIPOLYGON Z (((-76.45228 35.09324 0.00000, ..."
1,35.092890,-76.451971,NaN,Granite,"MULTIPOLYGON Z (((-76.45232 35.09293 0.00000, ..."
2,35.092743,-76.452027,NaN,Granite,"MULTIPOLYGON Z (((-76.45224 35.09274 0.00000, ..."
3,35.092548,-76.452037,NaN,Granite,"MULTIPOLYGON Z (((-76.45246 35.09263 0.00000, ..."
4,35.092352,-76.451922,NaN,Granite,"MULTIPOLYGON Z (((-76.45230 35.09245 0.00000, ..."
...,...,...,...,...,...
14352,35.093274,-76.452716,NaN,Marl,"MULTIPOLYGON Z (((-76.45295 35.09328 0.00000, ..."
14353,35.093175,-76.452674,NaN,Marl,"MULTIPOLYGON Z (((-76.45294 35.09318 0.00000, ..."
14354,35.093501,-76.452904,NaN,Marl,"MULTIPOLYGON Z (((-76.45313 35.09353 0.00000, ..."
14355,35.093656,-76.452866,NaN,Marl,"MULTIPOLYGON Z (((-76.45311 35.09367 0.00000, ..."
